## Importação dos pacotes

In [1]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [2]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)

## Carga dos dados

In [3]:
# carregar arquivo de dados de treino
data = pd.read_csv('weather-train.csv', index_col='datetime', parse_dates=['datetime'])

# mostrar alguns exemplos de registros
data.head()

,temperature
datetime,
2012-10-01 12:00:00,NaN
2012-10-01 13:00:00,10.8800
2012-10-01 14:00:00,10.9198
2012-10-01 15:00:00,11.0240
2012-10-01 16:00:00,11.1281


In [4]:
# quantas linhas e colunas existem?
data.shape

(36202, 1)

## Análise dos dados

In [5]:
# quais são as colunas e respectivos tipos de dados?
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36202 entries, 2012-10-01 12:00:00 to 2016-11-17 21:00:00
Data columns (total 1 columns):
temperature    36201 non-null float64
dtypes: float64(1)
memory usage: 565.7 KB


In [6]:
# existem colunas com dados nulos?
data[data.columns[data.isnull().any()]].isnull().sum()

temperature    1
dtype: int64

In [7]:
# sumário estatístico das características numéricas
data.describe().T

,count,mean,std,min,25%,50%,75%,max
temperature,36201.0,9.6287,11.2093,-29.6667,1.23,10.31,19.05,34.9


In [11]:
numeric_feats = data.dtypes[data.dtypes != "object"].index
numeric_feats

Index(['length', 'diameter', 'height', 'whole_weight', 'shucked_weight',
       'viscera_weight', 'shell_weight', 'rings'],
      dtype='object')

In [8]:
data.head(10).T

datetime,2012-10-01 12:00:00,2012-10-01 13:00:00,2012-10-01 14:00:00,2012-10-01 15:00:00,2012-10-01 16:00:00,2012-10-01 17:00:00,2012-10-01 18:00:00,2012-10-01 19:00:00,2012-10-01 20:00:00,2012-10-01 21:00:00
temperature,NaN,10.88,10.9198,11.024,11.1281,11.2323,11.3365,11.4407,11.5448,11.649


In [9]:
data.isna().sum()

temperature    1
dtype: int64